In [30]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from torch import nn
import torch
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt



In [31]:
train_dataset = datasets.CIFAR10(root = './dataset/', train = True, download = True, transform = transforms.ToTensor())
test_dataset = datasets.CIFAR10(root = './dataset/', train = False, download = True, transform = transforms.ToTensor() )

In [32]:
train_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)
data = next(iter(train_loader))[0]
mean = data.mean(dim=[0,2,3])
std  = data.std(dim=[0,2,3])

print("Mean:", mean)
print("Std:", std)


Mean: tensor([0.4914, 0.4822, 0.4465])
Std: tensor([0.2470, 0.2435, 0.2616])


In [33]:
# Augmentation

train_augment = transforms.Compose(
    [transforms.Resize((227, 227)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean, std = std)])

test_transform = transforms.Compose([
    transforms.Resize((227,227)),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean, std = std)
])

In [34]:
train_dataset = datasets.CIFAR10(root = './dataset/', train = True, download = True, transform = train_augment)
test_dataset = datasets.CIFAR10(root = './dataset/', train = False, download = True, transform = test_transform )

In [35]:
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 1000, shuffle = False) 

In [29]:
class BottleNeck(nn.Sequential):
    def __init__(self, in_channel, out_channel, downsample = False):
        super(BottleNeck, self).__init__()
        self.stride = 2 if downsample else 1
        self.conv1 = nn.Conv2d(in_channels = in_channel, out_channels = out_channel, kernel_size = 3, stride = self.stride, padding = 1)
        self.batch1 = nn.BatchNorm2d(out_channel)
        self.rel1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels = out_channel, out_channels = out_channel, kernel_size = 3, stride = 1, padding = 1)
        self.batch2 = nn.BatchNorm2d(out_channel)


        if downsample or in_channel != out_channel:
            self.conv = nn.Conv2d(in_channels = in_channel, out_channels = out_channel, kernel_size = 1, stride = self.stride, bias = False)
        else:
            self.conv = None
    def forward(self, x):
        residual = x
        if self.conv is not None:
            residual = self.conv(x)
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.rel1(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = x + residual
        x = self.rel1(x)
        return x 
        

class ResNet18(nn.Module):
    def __init__(self, in_channel = 3, classes = 10):
        super(ResNet18, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels= in_channel, out_channels = 64, kernel_size = 7 ,stride = 2, padding = 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        )
        self.layer_1 = nn.Sequential(
            BottleNeck(64,64, downsample = False),
            BottleNeck(64,64, downsample = False)
        )
        self.layer_2 = nn.Sequential(
            BottleNeck(64,128, downsample = True),
            BottleNeck(128,128, downsample = False)
        )
        self.layer_3 = nn.Sequential(
            BottleNeck(128,256, downsample = True),
            BottleNeck(256,256, downsample = False)
        )
        self.layer_4 = nn.Sequential(
            BottleNeck(256,512, downsample = True),
            BottleNeck(512,512, downsample = False)
        )
        # Final pooling and classifier
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))  # Reduces to 1x1 spatial size
        self.classifier = nn.Linear(512, classes)  # Maps features to classes

    def forward(self, x):
        x = self.features(x)
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = self.avg_pool(x)
        x = x.view(-1, 512)
        x = self.classifier(x)
        return x
        

In [36]:
device = "cuda"

In [38]:
model = ResNet18().to(device)


In [40]:
# Now set the parameters
epochs = 30
criterian = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)


In [ ]:
def evaluate(loader: DataLoader):
    test_true_label = []
    test_pred_label = []

    test_overall_loss = 0.0
    with torch.no_grad():
        for data, label in loader:
            data, label = data.to(device), label.to(device)
            test_prediction = model(data)
            
            test_loss = criterian(test_prediction, label)
            _,test_pred = torch.max(test_prediction,1)
            test_true_label.append(label.cpu().numpy())
            test_pred_label.append(test_pred.cpu().numpy())
            test_overall_loss += test_loss.item()
        average_test_loss = test_overall_loss/len(loader)
        ty_true = np.concatenate(test_true_label)
        ty_pred = np.concatenate(test_pred_label)
        testing_final_accuracy = accuracy_score(ty_true, ty_pred)
        return testing_final_accuracy, average_test_loss

def train(training_loader, testing_loader, test_eval: bool = True, patience: int  = 5):
    benchmark_testing_accuracy = 0.0
    benchmark_training_accuray = 0.0
    counter = 0
    train_plot_accuracy = []
    test_plot_accuracy = []
    train_plot_loss = []
    test_plot_loss = []
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        true_label = []
        pred_label = []
        for data, label in training_loader:
            data, label = data.to(device), label.to(device)
            model.zero_grad()
            train_pred = model(data)
            training_loss = criterian(train_pred, label)
            training_loss.backward()
            optimizer.step()
            _,prediction = torch.max(train_pred,1)
            true_label.append(label.cpu().numpy())
            pred_label.append(prediction.cpu().detach().numpy())
            train_loss += training_loss.item()
        average_loss = train_loss/len(training_loader)
        y_true = np.concatenate(true_label)
        y_pred = np.concatenate(pred_label)
        train_accuracy = accuracy_score(y_true, y_pred)

        testing_final_accuracy, average_test_loss = evaluate(testing_loader)
        train_plot_accuracy.append(train_accuracy)
        test_plot_accuracy.append(testing_final_accuracy)
        train_plot_loss.append(average_loss)
        test_plot_loss.append(average_test_loss)
        print(f'Epoch {epoch + 1}: Training Loss: {average_loss}, Training Accuracy: {train_accuracy}, Testing Loss: {average_test_loss}, Testing Accuracy: {testing_final_accuracy}')
        if testing_final_accuracy > benchmark_testing_accuracy:
            torch.save(model.state_dict(), './model_checkpoint/alex_net/alex_net.pth')
            benchmark_testing_accuracy = testing_final_accuracy
            counter = 0
            print(f"Model saved at acuracy of: {testing_final_accuracy}")
        else:
            counter += 1
        if counter > patience:
            print("Model stopping due to no improvement in testing accuracy")
            return train_plot_accuracy, test_plot_accuracy, train_plot_loss, test_plot_loss
    return train_plot_accuracy, test_plot_accuracy, train_plot_loss, test_plot_loss


In [43]:
train_plot_accuracy, test_plot_accuracy, train_plot_loss, test_plot_loss = train(train_loader, test_loader)

Epoch 1: Training Loss: 1.5903875054148457, Training Accuracy: 0.40706, Testing Loss: 1.1894361615180968, Testing Accuracy: 0.5774
Model saved at acuracy of: 0.5774
Epoch 2: Training Loss: 0.9856256225987163, Training Accuracy: 0.64914, Testing Loss: 0.7885006487369537, Testing Accuracy: 0.7191
Model saved at acuracy of: 0.7191
Epoch 3: Training Loss: 0.7311657948795792, Training Accuracy: 0.74412, Testing Loss: 0.705269992351532, Testing Accuracy: 0.756
Model saved at acuracy of: 0.756
Epoch 4: Training Loss: 0.6084909917372266, Training Accuracy: 0.78888, Testing Loss: 0.6135749161243439, Testing Accuracy: 0.7874
Model saved at acuracy of: 0.7874
Epoch 5: Training Loss: 0.5257273098773054, Training Accuracy: 0.81716, Testing Loss: 0.5461825430393219, Testing Accuracy: 0.8122
Model saved at acuracy of: 0.8122
Epoch 6: Training Loss: 0.46494248048271364, Training Accuracy: 0.8382, Testing Loss: 0.5108953028917312, Testing Accuracy: 0.8282
Model saved at acuracy of: 0.8282
Epoch 7: Trai